In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import tensorflow as tf
!pip install swifter
import swifter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for swifter: filename=swifter-1.4.0-py3-none-any.whl size=16505 sha256=b2f37b29fa43b2f7da1b3e03f18a1aeceee46175c4ac30683341e3822a884344
  Stored in directory: /root/.cache/pip/wheels/ef/7f/bd/9bed48f078f3ee1fa75e0b29b6e0335ce1cb03a38d3443b3a3
Successfully built swifter


In [14]:
df = pd.read_csv('/content/drive/MyDrive/Simple_RNN_Project/IMDB Dataset.csv')

In [4]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
!pip install sentence_transformer

ERROR: Could not find a version that satisfies the requirement sentence_transformer (from versions: none)
ERROR: No matching distribution found for sentence_transformer


In [6]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")  # 384-dim embeddings

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
def generate_embeddings(text):
  embedding = model.encode(text, convert_to_numpy = True)
  return embedding

In [15]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [16]:
df['review_encoded'] = df['review'].swifter.apply(generate_embeddings)

Pandas Apply:   0%|          | 0/50000 [00:00<?, ?it/s]

In [17]:
df['sentiment'] = df['sentiment'].map({'positive': 1, 'negative': 0})

In [20]:
df['review_encoded'].shape

(50000,)

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Dropout, Reshape

In [68]:
rnn_model = Sequential([SimpleRNN(384, activation = 'tanh', return_sequences = True, input_shape=(1, 384)),
                        Dropout(0.2),
                        SimpleRNN(128, activation = 'tanh'),
                        Dropout(0.2),
                        Dense(1, activation = 'sigmoid')])

rnn_model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn_9 (SimpleRNN)        │ (None, 1, 384)         │       295,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 1, 384)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_10 (SimpleRNN)       │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 361,089 (1.38 MB)

 Trainable params: 361,089 (1.38 MB)

 Non-trainable params: 0 (0.00 B)

In [71]:
rnn_model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [72]:
from sklearn.model_selection import train_test_split

In [73]:
X = df['review_encoded']
y = df['sentiment']

In [74]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [75]:
X_train.shape, y_train.shape

((40000,), (40000,))

In [76]:
X_train[0].shape

(384,)

In [77]:
X_test.shape, y_test.shape

((10000,), (10000,))

In [78]:
import numpy as np

X_train_np = np.array(X_train.tolist()).reshape(-1, 1, 384)
X_test_np = np.array(X_test.tolist()).reshape(-1, 1, 384)

rnn_model.fit(X_train_np, y_train, epochs=50, validation_data=(X_test_np, y_test), batch_size=32, validation_split=0.2)

Epoch 1/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.7746 - loss: 0.4690 - val_accuracy: 0.8146 - val_loss: 0.4059
Epoch 2/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.8071 - loss: 0.4149 - val_accuracy: 0.8159 - val_loss: 0.4052
Epoch 3/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8135 - loss: 0.4082 - val_accuracy: 0.8109 - val_loss: 0.4096
Epoch 4/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8156 - loss: 0.4059 - val_accuracy: 0.8054 - val_loss: 0.4176
Epoch 5/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.8139 - loss: 0.4069 - val_accuracy: 0.8199 - val_loss: 0.4009
Epoch 6/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8192 - loss: 0.3985 - val_accuracy: 0.8189 - val_loss: 0.3972
Epoch 7/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8119 - loss: 0.4066 - val_accuracy: 0.8184 - val_loss: 0.4002
Epoch 8/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8155 - loss: 0.3977 - 

In [88]:
import numpy as np

text = "Terrible!"
embedding = generate_embeddings(text)
# Reshape the embedding to match the input shape expected by the model
embedding = np.reshape(embedding, (1, 1, -1))
rnn_model.predict(embedding)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step


array([[0.00184978]], dtype=float32)

In [80]:
embedding.shape

(1, 1, 384)

In [91]:
rnn_model.save('/content/drive/MyDrive/Simple_RNN_Project/rnn_model.h5')

In [92]:
import tensorflow as tf
rnn_model.save('/content/drive/MyDrive/Simple_RNN_Project/rnn_model.keras', save_format='tf')  # Force TF/Keras 2 format

In [94]:
print(tf.__version__)

2.18.0


In [96]:
RNNMODEL = rnn_model.export("RNNMODEL.keras")

Saved artifact at 'RNNMODEL.keras'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 1, 384), dtype=tf.float32, name='keras_tensor_30')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  134023512409488: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134023512411792: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134023512411216: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134023512409872: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134023512412944: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134023512412368: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134023512411024: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134023512413328: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [97]:
import shutil

# Folder you want to zip
shutil.make_archive("RNNMODEL", 'zip', "RNNMODEL.keras")

'/content/RNNMODEL.zip'

In [102]:
print(rnn_model(embedding).numpy()[[0]])

[[0.00184978]]


In [103]:
print(rnn_model.signatures)

AttributeError: 'Sequential' object has no attribute 'signatures'